In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import glob
import multiprocessing

from operator import attrgetter
from sklearn.datasets import make_biclusters
from sklearn.datasets import samples_generator as sg
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import DBSCAN

from metrics import *

In [2]:
data = pd.read_csv('TestData/SimulatedDataCoherence/BiclusterABC_high.csv',header=None)
scaler = MinMaxScaler()
scaler.fit(data)
data = scaler.transform(data)*2-1

data = np.concatenate((data,data),axis=0)

In [3]:
class STSSCAN(object):
    def __init__(self, data):
        assert data is not None, "Empty data"
        self._data = data
        self._I, self._J = self._data.shape
            
    def fit(self):
        
            
        
        

SyntaxError: unexpected EOF while parsing (<ipython-input-3-518a2d7efca2>, line 11)